In [1]:
import urllib3
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
import urllib.request
import bs4 as bs
import re
from urllib.parse import urljoin

In [ ]:
url = 'https://sistemas.anm.gov.br/arrecadacao/extra/relatorios/arrecadacao_cfem.aspx'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_="tabelaRelatorio")
links = table.find_all('a')

# links
[l.get('href') for l in links]   
clean_links = [l for l in links if l.get('href') != None]

# construindo os links
relative_urls = [link.get('href') for link in clean_links]
estados_fimlinks = ['&uf=AC', '&uf=AL', '&uf=AM', '&uf=AP', '&uf=BA', '&uf=CE', '&uf=DF', '&uf=AC', '&uf=ES', '&uf=GO', '&uf=MA', '&uf=MG', '&uf=MS', '&uf=MT', '&uf=PA', '&uf=PB', '&uf=PE', '&uf=PI', '&uf=PR', '&uf=RJ', '&uf=RN', '&uf=RO', '&uf=RR', '&uf=RS', '&uf=SC', '&uf=SE', '&uf=SP', '&uf=TO']
link_inicio = 'https://sistemas.anm.gov.br/arrecadacao/extra/relatorios/'


#dados consolidado ano
consolidado_anos = pd.read_html(str(table))[0]
# dados consolidado mes

year = []
month = []
number = []

for response in soup.find_all("tr", class_='labelResultadoAlternado'):
    if response.get('id'):
        year_num = response.get('id')[2:6]
        month_list = response.find_all("td")
        
        month.append(month_list[0].text.split()[0])
        number.append(month_list[1].text.split()[0])
        year.append(year_num)
        
consolidado_mes = pd.DataFrame({"year": year, "month": month, "number": number})

# click the item 

url_anos = []
for i in range(0,len(relative_urls)):
    new_element = link_inicio + relative_urls[i]
    url_anos.append(new_element)


list_dfs = []

for i, x in zip(range(0,len(url_anos)),url_anos):
    url = link_inicio + relative_urls[i]
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(response.text, 'html.parser')
    table_estados = soup.find('table', class_="tabelaRelatorio")
    new_element = pd.read_html(str(table_estados))[0]
    list_dfs.append(new_element)
    
#caminhos das tabelas
url_anos = []

for i in range(0,len(relative_urls)):
    new_element = link_inicio + relative_urls[i]
    url_anos.append(new_element)

    
    
def urlfinal(lpossiveis, dpossiveis):
    uniao = []
    for l in lpossiveis:
        for d in dpossiveis:
            uniao.append(l + str(d))
    return uniao

url_anos_estado = urlfinal(url_anos, estados_fimlinks)

    
# Municípios


list_dfs_muncipio = []

for i, x in zip(range(0,len(url_anos_estado)),url_anos_estado):
    response = requests.get(url_anos_estado[i])
    html = response.content
    soup = BeautifulSoup(response.text, 'html.parser')
    table_estados_ind = soup.find('table', class_="tabelaRelatorio")
    new_element = pd.read_html(str(table_estados_ind))[0]
    list_dfs_muncipio.append(new_element)
    
from datetime import date

data_atual = date.today()



In [32]:
print('Data de extração:',data_atual) 
consolidado_anos.head()

Data de extração: 2020-10-10


,Ano/Mês,Total
0,2020 (Em Aberto),3.560.995.04379
1,2019,4.504.238.66890
2,2018,3.036.143.59241
3,2017,1.838.568.02145
4,2016,1.797.879.22675


In [33]:
print('Data de extração:',data_atual) 
print(consolidado_mes.head())

Data de extração: 2020-10-10
   year     month          number
0  2020   Outubro    4.437.647,57
1  2020  Setembro  534.503.638,94
2  2020    Agosto  490.275.930,98
3  2020     Julho  418.869.282,97
4  2020     Junho  406.750.435,15


In [35]:
for i, x in zip(range(0,len(url_anos)),url_anos):
    print('Data de extração:', data_atual) 
    print(url_anos[i])
    print(list_dfs[i].head())

Data de extração: 2020-10-10
https://sistemas.anm.gov.br/arrecadacao/extra/relatorios/arrecadacao_cfem_ano.aspx?ano=2020
  Estado         Jan.         Fev.         Mar.         Abr.         Mai.  \
0     AC      8.79173      5.35345     14.58834      3.69205      3.95305   
1     AL    237.74928    255.29415    213.19070    183.45236    226.81980   
2     AM    881.44511    963.27538    820.71849    680.88784    701.39030   
3     AP  1.658.49463  1.171.35107  1.062.84957  1.439.49957  1.104.95662   
4     BA  5.211.25452  5.056.05538  6.625.35550  7.900.34378  6.427.99590   

          Jun.         Jul.         Ago.         Set.      Out.  Nov.  Dez.  \
0      2.64760      3.14647      4.86916      3.80742     33484   NaN   NaN   
1    113.28971    146.83512    151.08253    194.86390      5915   NaN   NaN   
2    126.03345  1.070.87066    757.22212  1.127.28008   9.45090   NaN   NaN   
3  2.760.79599  2.924.34626  1.781.54743  1.897.23808   7.84430   NaN   NaN   
4  7.370.22005  9.016

In [36]:
for i, x in zip(range(0,len(url_anos_estado)),url_anos_estado):
    print('Data de extração:', data_atual) 
    print(url_anos_estado[i])
    print(list_dfs_muncipio[i].head())

Data de extração: 2020-10-10
https://sistemas.anm.gov.br/arrecadacao/extra/relatorios/arrecadacao_cfem_ano.aspx?ano=2020&uf=AC
  Estado         Jan.         Fev.         Mar.         Abr.         Mai.  \
0     AC      8.79173      5.35345     14.58834      3.69205      3.95305   
1     AL    237.74928    255.29415    213.19070    183.45236    226.81980   
2     AM    881.44511    963.27538    820.71849    680.88784    701.39030   
3     AP  1.658.49463  1.171.35107  1.062.84957  1.439.49957  1.104.95662   
4     BA  5.211.25452  5.056.05538  6.625.35550  7.900.34378  6.427.99590   

          Jun.         Jul.         Ago.         Set.      Out.  Nov.  Dez.  \
0      2.64760      3.14647      4.86916      3.80742     33484   NaN   NaN   
1    113.28971    146.83512    151.08253    194.86390      5915   NaN   NaN   
2    126.03345  1.070.87066    757.22212  1.127.28008   9.45090   NaN   NaN   
3  2.760.79599  2.924.34626  1.781.54743  1.897.23808   7.84430   NaN   NaN   
4  7.370.22005 

In [29]:
for i, x in zip(range(0,len(url_anos_estado)),url_anos_estado):
    print('Data de extração:') 
    print(data_atual)
    print(url_anos_estado[i])
    print(list_dfs_muncipio[i].head())

Data de extração:
2020-10-10
https://sistemas.anm.gov.br/arrecadacao/extra/relatorios/arrecadacao_cfem_ano.aspx?ano=2020&uf=AC
  Estado         Jan.         Fev.         Mar.         Abr.         Mai.  \
0     AC      8.79173      5.35345     14.58834      3.69205      3.95305   
1     AL    237.74928    255.29415    213.19070    183.45236    226.81980   
2     AM    881.44511    963.27538    820.71849    680.88784    701.39030   
3     AP  1.658.49463  1.171.35107  1.062.84957  1.439.49957  1.104.95662   
4     BA  5.211.25452  5.056.05538  6.625.35550  7.900.34378  6.427.99590   

          Jun.         Jul.         Ago.         Set.      Out.  Nov.  Dez.  \
0      2.64760      3.14647      4.86916      3.80742     33484   NaN   NaN   
1    113.28971    146.83512    151.08253    194.86390      5915   NaN   NaN   
2    126.03345  1.070.87066    757.22212  1.127.28008   9.45090   NaN   NaN   
3  2.760.79599  2.924.34626  1.781.54743  1.897.23808   7.84430   NaN   NaN   
4  7.370.22005 